<a href="https://colab.research.google.com/github/Suraez/laughing-octo-sniffle/blob/main/expt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is for converting the downloaded raw .txt data to .csv data, Ignore this if you have already got the CSV

In [1]:
import pandas as pd

# Specify the input TXT file path and output CSV file path
txt_file_path = "/content/data_2021_azure.txt"
csv_file_path = "/content/data.csv"

# Load the TXT file into a pandas DataFrame
# Assuming the TXT file is comma-separated
df = pd.read_csv(txt_file_path, sep=",")

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f"File converted and saved as {csv_file_path}")


File converted and saved as /content/data.csv


In [3]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!ls /content/drive

MyDrive  Shareddrives


In [5]:
!cd MyDrive

/bin/bash: line 1: cd: MyDrive: No such file or directory


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


In [8]:
import pandas as pd

# Load data
df = pd.read_csv('/content/drive/MyDrive/research/data.csv')

# Sort by app, func, and end_timestamp to ensure order
df = df.sort_values(by=['app', 'func', 'end_timestamp'])

# Calculate time interval in seconds between invocations for each function
df['time_interval'] = df.groupby(['app', 'func'])['end_timestamp'].diff()
df = df.dropna(subset=['time_interval'])  # Remove NaNs resulting from the first invocation


In [9]:
intervals = df['time_interval'].values


In [10]:
sequence_length = 10  # Adjust as needed
X = np.lib.stride_tricks.sliding_window_view(intervals, sequence_length)
y = intervals[sequence_length:]

In [11]:
# Reshape X for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshaping to (samples, sequence_length, features)
# Ensure X and y have consistent lengths
X = X[:-1]


### Normalization of X and Y

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X.reshape(-1, 1)).reshape(X.shape)
y = scaler.fit_transform(y.reshape(-1, 1))


In [13]:
# Split into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# Define the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),  # Regularization to prevent overfitting
    Dense(50, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mse')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2,
                    callbacks=[early_stopping], verbose=1)


Epoch 1/5
39611/39611 ━━━━━━━━━━━━━━━━━━━━ 304s 8ms/step - loss: 2.6688e-05 - val_loss: 2.2156e-05
Epoch 2/5
39611/39611 ━━━━━━━━━━━━━━━━━━━━ 301s 8ms/step - loss: 2.6738e-05 - val_loss: 2.1593e-05
Epoch 3/5
21558/39611 ━━━━━━━━━━━━━━━━━━━━ 2:07 7ms/step - loss: 2.0887e-05

KeyboardInterrupt: 

In [16]:
# Example usage
app_id = '85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031'  # Replace with the actual app ID
func_id = 'e02465de583b6ceffa5b78cce5f10eb27e714a8a6b3aed483be50f30a924071f'  # Replace with the actual function ID
next_invocation, predicted_interval = predict_next_invocation(model, X, df, app_id, func_id)
print(f"Predicted next invocation timestamp: {next_invocation}")
print(f"Predicted time interval: {predicted_interval}")

NameError: name 'predict_next_invocation' is not defined

In [17]:
!git clone https://github.com/Suraez/laughing-octo-sniffle.git
%cd laughing-octo-sniffle
!git status


Cloning into 'laughing-octo-sniffle'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 4.92 KiB | 4.92 MiB/s, done.
/content/laughing-octo-sniffle
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
